In [87]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


D:\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['step']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [ ]:
def gwp(x,xc,pc,alpha,gamma):
    """Gaussian wavepacket in one dimension."""
    
    return (2/pi*real(alpha))**0.25 * exp(-alpha*(x-xc)**2 + 1j*pc*(x-xc) + 1j*gamma)

In [3]:
from qdyn import propagator, animate_dynamics


In [4]:
def V_ground(x):
    de=0.19158
    re=2.29716
    beta=0.21098
    te=0.1
    return de*(1-re/x*np.exp(-beta*(x**2-re**2)))**2+te


def V_excited(x):
    de=0.03342
    re=3.03150
    beta=0.19459
    te=0.327
    return de*(1-re/x*np.exp(-beta*(x**2-re**2)))**2+te

In [176]:
x=np.linspace(0,8,200)
plt.plot(x,V_ground(x))
plt.ylim([0,0.6])

C:\Users\AlbertCielstian\AppData\Local\Temp\ipykernel_7944\1647881167.py:6: RuntimeWarning: divide by zero encountered in true_divide
  return de*(1-re/x*np.exp(-beta*(x**2-re**2)))**2+te


(0.0, 0.6)

In [5]:
def k_grnd():
    de=0.19158
    re=2.29716
    beta=0.21098
    te=0.1
    k_grnd = 2*de*(1/re+2*beta*re)**2
    return k_grnd

In [75]:
from numpy.polynomial.hermite import hermval

def eigen_ho(x,v,m,k):
    """Calculates the eigenfunction of the harmonic oscillator system.
    
    Arguments
    x: is a space coordinate.
    v: is the vibrational quantum number.
    m: is the mas of the system.
    k: is the force constant of the harmonic potential.
    """
    
    hermite_sum=zeros(v+1)
    hermite_sum[-1]=1
    return 1/(2**v * math.factorial(v))**0.5 * (((m*k)**0.5)/pi)**0.25 * np.e**(-x**2 * ((m*k)**0.5)/2) * hermval((m*k)**0.25 * x,hermite_sum) 

In [178]:
m=14583 #reduced mass of O2 in atomic units
dt=10 #time step in atomic units (this is a relatively big step)

x_grid=np.linspace(1,6,500)
psi0 = eigen_ho((x_grid-2.29716),0,m,k_grnd())
new_wf=propagator((x_grid-2.29716),psi1,m,dt,V_ground)


plt.plot(x_grid,psi0)
plt.plot(x_grid,V_ground(x_grid))

In [181]:
dt=0.001
nsteps=500

x_grid=np.linspace(1,6,500)
psi0=eigen_ho((x_grid-2.29716),5,m,k_grnd())

wf_dynamics=zeros((nsteps+1,len(x_grid)),dtype=complex128)
wf_dynamics[0]=psi0

for step in range(nsteps):
    psi=propagator((x_grid-2.29716),wf_dynamics[step],m,dt,V_ground)
    wf_dynamics[step+1]=psi

animate_dynamics(x_grid,wf_dynamics,dt,V_ground(x_grid))

In [93]:
trapz(wf_dynamics[0],wf_dynamics[0])

(-2.3697074988954016e-54+0j)

In [ ]:
conj(wf_dynamics[0])*wf_dynamics

In [98]:
import numpy as np

def TDM(x):
    '''Transition dipole moment dependence on the inter-atomic
    coordinate x for the O2 electronic transition.
    The input x, and the output of the function are in atomic units.'''

    #parameters for connecting function
    ss=2
    es=2.28
    plateau=0.266
    peak=0.94

    #step function of order 7 (n=3)
    xs=(x-ss)/(es-ss)
    step= -20*xs**7 + 70*xs**6 - 84*xs**5 + 35*xs**4
    step= plateau + (peak-plateau)*step

    #exponential decay for higher x values
    decay=3.85*np.exp(-0.626*x)

    return np.where(x<ss,plateau,np.where(x>es,decay,step))

In [165]:
m=14583 #reduced mass of O2 in atomic units
dt=1
nsteps=500

x_grid=np.linspace(1,6,500)

wf_dynamics=zeros((nsteps+1,len(x_grid)),dtype=complex128)
wf_dynamics[0]=psi0

for step in range(nsteps):
    psi=propagator((x_grid-2.29716),wf_dynamics[step],m,dt,V_ground)
    wf_dynamics[step+1]=psi

    
animate_dynamics(x_grid,wf_dynamics,dt,V_ground(x_grid))


In [174]:
m=14583 #reduced mass of O2 in atomic units
dt=0.00001
nsteps=50000

x_grid=np.linspace(1,6,500)

psi0=eigen_ho((x_grid-2.29716),0,m,k_grnd())
wf_dynamics=zeros((nsteps+1,len(x_grid)),dtype=complex128)
wf_dynamics[0]=psi0

for step in range(nsteps):
    psi=propagator((x_grid-2.29716),wf_dynamics[step],m,dt,V_ground)
    wf_dynamics[step+1]=psi

    

psi0_exi=eigen_ho((x_grid-3.03150),0,m,k_grnd())
wf_dynamics_exi=zeros((nsteps+1,len(x_grid)),dtype=complex128)
wf_dynamics_exi[0]=psi0_exi

for step in range(nsteps):
    psi_exi=propagator((x_grid-2.29716),wf_dynamics_exi[step],m,dt,V_excited)
    wf_dynamics_exi[step+1]=psi_exi    
    
#animate_dynamics(x_grid,wf_dynamics,dt,V_ground(x_grid))



autocor_int = trapz(conj(wf_dynamics[0])*wf_dynamics_exi[1:],x_grid)



# Obtain the inverse Fourier transform of gauss_t. Note that the values are shifted by fftshit()
gauss_w=ifftshift(ifft(autocor_int,norm="ortho")) 
spec = (2*pi)**(3/2)/(3*137.037)*gauss_w * TDM(2.29716)**2



# Obtain the grid of frequency values. The d keyword specifies the spacing between time points.
frequency=2 * pi * fftshift(fftfreq(nsteps,d=dt))

spec_frq = spec*frequency

plot(frequency,abs(spec_frq)) # Plot the transformed function
xlim([0,1500])
show()

In [182]:
psi0

array([-2.16824254e-033, -8.11833847e-033, -3.00685102e-032,
       -1.10164053e-031, -3.99253615e-031, -1.43131897e-030,
       -5.07574997e-030, -1.78048278e-029, -6.17798108e-029,
       -2.12043040e-028, -7.19892196e-028, -2.41754283e-027,
       -8.03047723e-027, -2.63855483e-026, -8.57522506e-026,
       -2.75661762e-025, -8.76508247e-025, -2.75664443e-024,
       -8.57526324e-024, -2.63847352e-023, -8.02958488e-023,
       -2.41693474e-022, -7.19555885e-022, -2.11879087e-021,
       -6.17066093e-021, -1.77742486e-020, -5.06363765e-020,
       -1.42672898e-019, -3.97578962e-019, -1.09573056e-018,
       -2.98660565e-018, -8.05083327e-018, -2.14628591e-017,
       -5.65865068e-017, -1.47540540e-016, -3.80432349e-016,
       -9.70075020e-016, -2.44618350e-015, -6.09989529e-015,
       -1.50417700e-014, -3.66785423e-014, -8.84412462e-014,
       -2.10872594e-013, -4.97164938e-013, -1.15901269e-012,
       -2.67162323e-012, -6.08910142e-012, -1.37218763e-011,
       -3.05736333e-011,

In [183]:
m=14583 #reduced mass of O2 in atomic units
dt=0.00001
nsteps=5000

x_grid=np.linspace(1,6,500)

psi0=eigen_ho((x_grid-2.29716),0,m,k_grnd())
wf_dynamics=zeros((nsteps+1,len(x_grid)),dtype=complex128)
wf_dynamics[0]=psi0



psi0_exi=eigen_ho((x_grid-3.03150),0,m,k_grnd())
wf_dynamics_exi=zeros((nsteps+1,len(x_grid)),dtype=complex128)
wf_dynamics_exi[0]=psi0_exi

for step in range(nsteps):
    psi_exi=propagator((x_grid-2.29716),wf_dynamics_exi[step],m,dt,V_excited)
    wf_dynamics_exi[step+1]=psi_exi    
    
#animate_dynamics(x_grid,wf_dynamics,dt,V_ground(x_grid))



autocor_int = trapz(conj(wf_dynamics[0])*wf_dynamics_exi[1:],x_grid)



# Obtain the inverse Fourier transform of gauss_t. Note that the values are shifted by fftshit()
gauss_w=ifftshift(ifft(autocor_int,norm="ortho")) 
spec = (2*pi)**(3/2)/(3*137.037)*gauss_w * TDM(2.29716)**2



# Obtain the grid of frequency values. The d keyword specifies the spacing between time points.
frequency=2 * pi * fftshift(fftfreq(nsteps,d=dt))

spec_frq = spec*frequency

plot(frequency,abs(spec_frq)) # Plot the transformed function
xlim([0,1500])
show()

In [157]:
listT = []
ans=0

for i in range(500):
    ans += 0.001
    listT.append(ans)

plot(listT,abs(autocor_int)) # Plot the transformed function
show()


In [154]:
listT

[0.001,
 0.002,
 0.003,
 0.004,
 0.005,
 0.006,
 0.007,
 0.008,
 0.009000000000000001,
 0.010000000000000002,
 0.011000000000000003,
 0.012000000000000004,
 0.013000000000000005,
 0.014000000000000005,
 0.015000000000000006,
 0.016000000000000007,
 0.017000000000000008,
 0.01800000000000001,
 0.01900000000000001,
 0.02000000000000001,
 0.02100000000000001,
 0.022000000000000013,
 0.023000000000000013,
 0.024000000000000014,
 0.025000000000000015,
 0.026000000000000016,
 0.027000000000000017,
 0.028000000000000018,
 0.02900000000000002,
 0.03000000000000002,
 0.03100000000000002,
 0.03200000000000002,
 0.03300000000000002,
 0.03400000000000002,
 0.035000000000000024,
 0.036000000000000025,
 0.037000000000000026,
 0.03800000000000003,
 0.03900000000000003,
 0.04000000000000003,
 0.04100000000000003,
 0.04200000000000003,
 0.04300000000000003,
 0.04400000000000003,
 0.04500000000000003,
 0.046000000000000034,
 0.047000000000000035,
 0.048000000000000036,
 0.04900000000000004,
 0.050000000